<a href="https://colab.research.google.com/github/ipeirotis/sql_autograding/blob/main/Few_shot_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install -q openai
!pip install -q google-cloud-secret-manager
!pip3 install -U -q PyMySQL sqlalchemy sql_magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 739.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 5.1 MB/s eta 0:00:00


In [ ]:
from google.colab import auth

# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [ ]:
from google.cloud import secretmanager
import os

def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload of the given secret version and return it.

    Args:
        project_id (str): Google Cloud project ID.
        secret_id (str): ID of the secret to access.
        version_id (str): ID of the version to access.
    Returns:
        str: The secret version's payload, or None if
        the version does not exist.
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")


openai_key = access_secret_version("sql-autograding", "openai-gpt4-32k", "latest")

In [ ]:
import openai
openai.api_key = openai_key

In [ ]:
from google.colab import files
import io
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import gcsfs
# load the cleaned dataset
fs = gcsfs.GCSFileSystem(project="sql_autograding")
with fs.open("gs://sql_autograding/clean_submissions_final.csv") as f:
    data = pd.read_csv(f)

In [ ]:
data = data.drop(columns="Unnamed: 0")
data.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback
0,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969729,Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \r\n A.CustomerID,\r\n A.CompanyN...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (SELECT C.CustomerID, C.Co...",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...
1,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969713,"For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\r\n-1 pts for using...,"SELECT O.OrderID, O.CustomerID&#160; &#160; , ...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...
2,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969711,We want to identify the ship countries with th...,Long Answer Question,"use northwind;\r\nSELECT \r\n ShipCountry, ...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n..."
3,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969710,For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN
4,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969709,Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\r\nSELECT \r\n min(OrderDate...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDateFROM...,True,"['northwind', 'flights']",7.5,NaN


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text

## Further preprocessing

In [ ]:
db_list = [
    "flights",
    "imdb",
    "music",
    "restaurants",
    "facebook",
    "northwind",
    "collisions",
]


def db_schema(db_name):
    """
    Connects to a database and returns the schema of each table in the database.

    The function connects to a specific database using SQLAlchemy. It then retrieves the list of tables in
    the database and for each table, it queries the schema (i.e., the list of fields/columns) and stores it
    in a dictionary. The function returns a list of such dictionaries, with each dictionary representing a table
    and its corresponding schema.

    Args:
        db_name (str): The name of the database to connect to and retrieve schemas from.

    Returns:
        list: A list of dictionaries, with each dictionary containing the name of a table as the key and a
              list of its fields as the value.

    Example:
        >>> db_schema('flights')
        {'m_airports': ['airport', 'state', 'state_name']},
        {'m_ticket_prices': ['origin', 'dest', 'carrier', 'fare', 'fare_per_mile','passengers', 'distance']
        ...
        },
    """

    student_password = access_secret_version(
        "sql-autograding", "db_student_password", "1"
    )

    conn_string = (
        "mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4".format(
            host="db.ipeirotis.org",
            user="student",
            password=student_password,
            db=db_name,
            encoding="utf8mb4",
        )
    )
    engine = create_engine(conn_string)

    with engine.begin() as conn:
        tables = pd.read_sql_query(sql=text("show tables"), con=conn)
        tables = tables.iloc[:, -1].tolist()

        schema_list = []
        for t in tables:
            d = pd.read_sql_query(sql=text(f"describe {t}"), con=conn)
            table_schema = {t: d.loc[:, "Field"].tolist()}
            schema_list.append(table_schema)
    return schema_list

In [ ]:
# generates the schema for each of the databases in the dataset
flights_schema = db_schema("flights")
imdb_schema = db_schema("imdb")
music_schema = db_schema("music")
restaurants_schema = db_schema("restaurants")
facebook_schema = db_schema("facebook")
northwind_schema = db_schema("northwind")
collisions_schema = db_schema("collisions")

print(facebook_schema)

[{'Concentration': ['ProfileID', 'Concentration']}, {'FavoriteBooks': ['ProfileID', 'Book']}, {'FavoriteMovies': ['ProfileID', 'Movie']}, {'FavoriteMusic': ['ProfileID', 'Music']}, {'FavoriteTVShows': ['ProfileID', 'TVShow']}, {'Hobbies': ['ProfileID', 'Hobby']}, {'LookingFor': ['ProfileID', 'LookingFor']}, {'Orientation': ['ProfileID', 'InterestedIn']}, {'Profiles': ['ProfileID', 'Name', 'MemberSince', 'LastUpdate', 'School', 'Status', 'Sex', 'Birthday', 'AIM', 'Website', 'PoliticalViews', 'Geography', 'HighSchool', 'HomeTown', 'HomeState', 'Residence', 'CurrentAddress', 'CurrentTown', 'CurrentState']}, {'Relationship': ['ProfileID', 'Status']}, {'chart': ['music', 'gender', 'cnt', 'music_rank']}, {'popular_music': ['music', 'cnt']}, {'popular_music_by_sex': ['music', 'gender', 'cnt']}]


In [ ]:
# creates a dictionary that maps each schema to each database and add the schema as a new column in the dataframe
schema_mapping = {
    "flights": flights_schema,
    "imdb": imdb_schema,
    "music": music_schema,
    "restaurants": restaurants_schema,
    "facebook": facebook_schema,
    "northwind": northwind_schema,
    "collisions": collisions_schema,
    "['northwind', 'flights']": (northwind_schema, flights_schema),
}

data["schema"] = data["Database"].map(schema_mapping)
data.head()

,Unnamed: 0,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
0,1,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969729,Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \r\n A.CustomerID,\r\n A.CompanyN...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (SELECT C.CustomerID, C.Co...",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'..."
1,3,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969713,"For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\r\n-1 pts for using...,"SELECT O.OrderID, O.CustomerID&#160; &#160; , ...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'..."
2,4,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969711,We want to identify the ship countries with th...,Long Answer Question,"use northwind;\r\nSELECT \r\n ShipCountry, ...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'..."
3,5,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969710,For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."
4,6,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969709,Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\r\nSELECT \r\n min(OrderDate...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDateFROM...,True,"['northwind', 'flights']",7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."


In [ ]:
from bs4 import BeautifulSoup

# Create a function to clean up a single text string
def clean_html_content(text):
    if pd.isna(text):
        return text
    if not isinstance(text, str):
        text = str(text)
    try:
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()
    except:
        return text


# Apply the function to every text column in your DataFrame
for col in data.columns:
    if col in ("Database", "schema"):
        continue
    if data[col].dtype == object:  # if the column is a text column
        data[col] = data[col].apply(clean_html_content)

<ipython-input-15-f8217fd0dcfa>:10: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


## Generating Mega Prompt
===============================

a. We are working with the XXX database, which has the following tables: artist(id, first_name, last_name) album(id, artist_id, name) track....

b. The request to the student is "Fetch all the tracks for user X"

c. The model answer is ..... (note that we may have multiple correct answers, the model answer is just an example)

d. The student answer was .....

e. Previously, students have submitted these answers and got back these responses and grades:

e1. submission: ..... , grade: ..... , feedback

e2. submission: ..... , grade: ..... , feedback

e3. submission: ..... , grade: ..... , feedback

Please provide a grade and feedback for the student

===============================

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from typing import List


def generate_submissions_string(
    train_df: pd.DataFrame, submission_size: int = None
) -> str:
    """
    Generate a string that concatenates student submissions up to the given size.

    Args:
        train_df (pd.DataFrame): The DataFrame containing the training data.
        submission_size (int, optional): The number of submissions to include in the string. If None, include all submissions.

    Returns:
        str: A string containing the concatenated submissions.
    """
    submissions = ""
    for i, row in train_df.iterrows():
        if submission_size is not None and i >= submission_size:
            break
        submission = row["InputUserAnswer"]
        grade = row["Score"]
        full_grade = row["full_score"]
        feedback = row["feedback"]
        submissions += f"e{i}. submission: {submission}, grade: {grade}/{full_grade}, feedback: {feedback} \\n\\n"
    return submissions


def generate_mega_prompt_for_id(
    df: pd.DataFrame, id: str, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        id (str): The QuestionId to generate the mega prompt for.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = df[df["QuestionId"] == id]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]

      test = temp.loc[0, "InputUserAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade (out of 10 points) and feedback for the student
        """
      return mega_p

    # making test size 1
    train_df, test_df = train_test_split(temp, test_size=1, random_state=1234)
    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    submissions = generate_submissions_string(train_df, submission_size)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]

    test = test_df.loc[0, "InputUserAnswer"]

    mega_p = f"""
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

      Please provide a grade and feedback for the student
      """
    return mega_p


def generate_mega_prompts(
    df: pd.DataFrame, submission_size: int = None
) -> pd.DataFrame:
    """
    Generate a DataFrame of mega prompts for each unique QuestionId in the given DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        submission_size (int, optional): The number of submissions to include in each mega prompt. If None, include all submissions.

    Returns:
        pd.DataFrame: A DataFrame containing the mega prompts.
    """
    result = []
    qids = df["QuestionId"].unique()

    for id in qids:
        mega_p = generate_mega_prompt_for_id(df, id, submission_size)
        new_row = {"QuestionId": id, "MegaPrompt": mega_p}
        result.append(new_row)

    mega_df = pd.DataFrame(result)

    return mega_df

In [ ]:
# for all questions each with all submissions => new df
mega_df_all = generate_mega_prompts(data)
mega_df_all.head()

,QuestionId,MegaPrompt
0,4969729,\n a. We are working with the ['northwind...
1,4969713,\n a. We are working with the ['northwind...
2,4969711,\n a. We are working with the ['northwind...
3,4969710,\n a. We are working with the ['northwind...
4,4969709,\n a. We are working with the ['northwind...


In [ ]:
# Answering Mega Prompt Using GPT 3.5 model
# Deal with rate limit of GPT 3.5-turbo model
from tenacity import(retry, stop_after_attempt, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT3_generation_with_backoff(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
# Answering Mega Prompt Using GPT 4 model
def GPT4_generation(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
# A function that takes QuestionId and submission size as input, generating n-size mega prompt with a random test case and printing out GPT4 answer
def test_shots(qid, n):
    df = data[data["QuestionId"] == qid]
    df_n = generate_mega_prompts(df, n)
    print(f"{n} submissions:")
    print(GPT4_generation(df_n.loc[0, "MegaPrompt"]))

In [ ]:
mega_df_all['QuestionId'].unique()

array([4969729, 4969713, 4969711, 4969710, 4969709, 4969708, 4969707,
       4969706, 4969705, 5119793, 5119791, 5119790, 5119787, 5119726,
       5119725, 5119723, 5119721, 5119720, 5119719, 5119651, 5119650,
       5119649, 5119648, 5119647, 5119646, 5119736, 5119735, 5119734,
       5119733, 5119732, 5119731, 5119730, 5119729, 5119728, 5119727,
       5119680, 5119679, 5119678, 5119677, 5119676, 5119675, 5119674,
       5119673, 5119653, 4969714, 4956678, 4956293, 4956290, 4338658,
       5168443, 5168442, 5168441, 5168440, 5168439, 5168438, 5168437,
       5119657, 5119656, 5119655, 5119654, 5117076, 5117074, 5117073,
       5117072, 5117071, 5117070, 5119652])

In [ ]:
# for i in range(0, 21, 5):
#     test_shots(5119650, i)

0 submissions:
The student's answer is correct.

Grade: 10/10

Feedback: Great job! Your SQL query correctly retrieves the artist_id and album_name (renamed as album_title) from the album table, as requested. Keep up the good work!
5 submissions:
Grade: 1.0/1.0

Feedback: Your answer is correct. You have successfully selected the artist_id and album_name (renamed as album_title) from the album table, and you have correctly not included the album_id in your query. Good job!
10 submissions:
The student's answer is correct. 

Grade: 1.0/1.0

Feedback: Good job! Your SQL query correctly selects the artist_id and album_name (renamed as album_title) from the album table, as requested.
15 submissions:
The student's answer is correct. 

Grade: 1.0/1.0

Feedback: Your SQL query correctly selects the artist_id and album_name (renamed as album_title) from the album table. Good job!
20 submissions:
The student's answer is correct. 

Grade: 1.0/1.0

Feedback: Your SQL query correctly selects the ar

In [ ]:
from tqdm import tqdm

In [ ]:
# Generate GPT4 answers for all QuestionIds (if needed)
for i, p in tqdm(mega_df_all.iterrows()):
    p["feedback"] = GPT4_generation(p["MegaPrompt"])

mega_df_all.head()

In [ ]:
mega_df_all["feedback"] = mega_df_all["MegaPrompt"].apply(GPT4_generation)
mega_df_all.head()

## DataFrame with Zero-Shot Prompts

In [ ]:
data1 = data.sample(n=800, random_state=1234).reset_index()
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",1.0,xs2262,NaN,"SELECT track_name, artist_name, album_name\r\n...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",1.0,ty2394,NaN,"SELECT artist_id, album_id, SUM(time) AS total...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",5.0,bak8119,NaN,"SELECT ProductID, ProductNameFROM Products WHE...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,1.0,lj2371,NaN,\r\nSELECT *\r\nFROM track\r\nWHERE artist_id ...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",5.0,fc2187,NaN,"SELECT FirstName, LastName, HireDate FROM Empl...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."


In [ ]:
# A function that parses grade and feedback from GPT responses
def parse_grade_feedback(text):
  colon_pos = text.find(":")
  divider_pos = text.find("/")

  grade = text[colon_pos+2:divider_pos]

  feedback_start = text.find("Feedback: ")
  if feedback_start != -1:
      feedback = text[feedback_start + len("Feedback: "):]
  else:
      feedback = None
  return grade, feedback

In [ ]:
lst = []
for i,r in data1.iterrows():
  # generate zero-shot mega prompt for each row of the dataframe
  lst.append(generate_mega_prompts(r.to_frame().T,0).loc[0,'MegaPrompt'])
# append mega prompts as a new column to the dataframe
data1['0_mp'] = lst
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema,0_mp
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",1.0,xs2262,NaN,"SELECT track_name, artist_name, album_name\r\n...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",1.0,ty2394,NaN,"SELECT artist_id, album_id, SUM(time) AS total...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",5.0,bak8119,NaN,"SELECT ProductID, ProductNameFROM Products WHE...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,1.0,lj2371,NaN,\r\nSELECT *\r\nFROM track\r\nWHERE artist_id ...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",5.0,fc2187,NaN,"SELECT FirstName, LastName, HireDate FROM Empl...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...


In [ ]:
tqdm.pandas()
# generate GPT4-32k responses for each row and parse grade and feedback as new columns
data1['4_0_answer'] = data1['0_mp'].progress_apply(GPT4_generation_with_backoff)
data1['4_0_grade'], data1['4_0_feedback'] = zip(*data1['4_0_answer'].apply(parse_grade_feedback))
data1.head()

100%|██████████| 800/800 [2:05:23<00:00,  9.40s/it]


,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,AnswerKey,isQuery,Database,full_score,feedback,schema,0_mp,4_0_answer,4_0_grade,4_0_feedback
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",...,"SELECT track_name, artist_name, album_name\r\n...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query correctly retrieves ...
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",...,"SELECT artist_id, album_id, SUM(time) AS total...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...,Grade: 10/10\n\nFeedback: Great job! Your quer...,10,Great job! Your query correctly identifies the...
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",...,"SELECT ProductID, ProductNameFROM Products WHE...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query is correct and it wi...
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,...,\r\nSELECT *\r\nFROM track\r\nWHERE artist_id ...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...,Grade: 10/10\n\nFeedback: Great job! Your quer...,10,Great job! Your query correctly retrieves the ...
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",...,"SELECT FirstName, LastName, HireDate FROM Empl...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query is correct and it me...


In [ ]:
data1.to_csv('zero_shot_4.csv')

In [ ]:
fs = gcsfs.GCSFileSystem(project="sql_autograding")
with fs.open("gs://sql_autograding/zero_shot_4.csv") as f:
    data1 = pd.read_csv(f)

In [ ]:
# generate GPT3.5-turbo responses for each row and parse grade and feedback as new columns
tqdm.pandas()
data1['3_0_answer'] = data1['0_mp'].progress_apply(GPT3_generation_with_backoff)
data1['3_0_grade'], data1['3_0_feedback'] = zip(*data1['3_0_answer'].apply(parse_grade_feedback))
data1.head()

100%|██████████| 800/800 [1:21:43<00:00,  6.13s/it]


,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,full_score,feedback,schema,0_mp,4_0_answer,4_0_grade,4_0_feedback,3_0_answer,3_0_grade,3_0_feedback
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",...,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10.0,Great job! Your SQL query correctly retrieves ...,Grade: 8/10\n\nFeedback: \n- Your query correc...,8,"\n- Your query correctly joins the track, albu..."
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",...,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...,Grade: 10/10\n\nFeedback: Great job! Your quer...,10.0,Great job! Your query correctly identifies the...,Grade: 7/10\n\nFeedback: \n- The student corre...,7,\n- The student correctly identified the table...
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",...,5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10.0,Great job! Your SQL query is correct and it wi...,Grade: 9/10\n\nFeedback: \nGreat job! Your que...,9,\nGreat job! Your query is correct and retriev...
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,...,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music dat...,Grade: 10/10\n\nFeedback: Great job! Your quer...,10.0,Great job! Your query correctly retrieves the ...,Grade: 8/10\n\nFeedback: \nThe student's answe...,8,\nThe student's answer is almost correct. They...
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",...,5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10.0,Great job! Your SQL query is correct and it me...,Grade: 9/10\n\nFeedback: \nGreat job on provid...,9,\nGreat job on providing the correct query to ...


In [ ]:
# save the dataframe with zero-shot mega prompts and GPT responses for evaluation
data1.to_csv('zero_shots_completion_800.csv')

## Mega-Prompt Generation

In [ ]:
# A function that generates few-shot mega-prompts for a row given the expected number of submissions
from numpy import row_stack
def generate_fewshot_prompt_for_row(
    test, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        test (pd.DataFrame): The DataFrame - this row.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = data[data["QuestionId"] == test['QuestionId']]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]

      test = temp.loc[0, "InputUserAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade (out of 10 points) and feedback for the student
        """
      return mega_p

    # skip the current row
    temp = temp[temp['NetId']!=test['NetId']]
    if submission_size==None:
      train_df = temp.copy().reset_index()
    else:
      # randomly sampling certain size of training rows for the same QuestionId
      train_df = temp.sample(n=submission_size).reset_index()

    submissions = generate_submissions_string(train_df)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]
    test = test['InputUserAnswer']

    mega_p = f"""
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

      Please provide a grade and feedback for the student.

      """ + """Only provide a RFC8259 compliant JSON response following this format without deviation. {"grade": "the grade out of 10 you give", "feedback": "the feedback you give"}"""
    return mega_p


## Sample Records from Original DF

In [ ]:
# Randomly select 300 rows from the dataframe
data1 = data.sample(n=300, random_state=1234).reset_index()
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",1.0,xs2262,NaN,"SELECT track_name, artist_name, album_name\r\n...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",1.0,ty2394,NaN,"SELECT artist_id, album_id, SUM(time) AS total...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",5.0,bak8119,NaN,"SELECT ProductID, ProductNameFROM Products WHE...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,1.0,lj2371,NaN,\r\nSELECT *\r\nFROM track\r\nWHERE artist_id ...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",5.0,fc2187,NaN,"SELECT FirstName, LastName, HireDate FROM Empl...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."


In [ ]:
# Generate few-shot mega prompts with submission size = all for GPT4-32k model
lst = []
for i,r in data1.iterrows():
  lst.append(generate_fewshot_prompt_for_row(r))

data1['4_few_mp'] = lst
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema,4_few_mp
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",1.0,xs2262,NaN,"SELECT track_name, artist_name, album_name\r\n...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",1.0,ty2394,NaN,"SELECT artist_id, album_id, SUM(time) AS total...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",5.0,bak8119,NaN,"SELECT ProductID, ProductNameFROM Products WHE...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,1.0,lj2371,NaN,\r\nSELECT *\r\nFROM track\r\nWHERE artist_id ...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",5.0,fc2187,NaN,"SELECT FirstName, LastName, HireDate FROM Empl...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...


In [ ]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.5 MB/s eta 0:00:00


In [ ]:
import tiktoken

# counting the number of tokens of mega-prompts
def num_tokens_from_string(string: str, encoding_name="cl100k_base") -> int:
  encoding = tiktoken.get_encoding(encoding_name)
  num_tokens = len(encoding.encode(string))
  return num_tokens

In [ ]:
# Counting the number of tokens for each mega prompt
data1['num'] = data1['4_few_mp'].apply(num_tokens_from_string)
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema,4_few_mp,num
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",...,xs2262,NaN,"SELECT track_name, artist_name, album_name\r\n...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,16322
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",...,ty2394,NaN,"SELECT artist_id, album_id, SUM(time) AS total...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,11975
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",...,bak8119,NaN,"SELECT ProductID, ProductNameFROM Products WHE...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,9889
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,...,lj2371,NaN,\r\nSELECT *\r\nFROM track\r\nWHERE artist_id ...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,8637
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",...,fc2187,NaN,"SELECT FirstName, LastName, HireDate FROM Empl...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,11330


In [ ]:
print(data1['num'].max())

28923


In [ ]:
from tqdm import tqdm
import json

In [ ]:
# a function that takes the current row and maximum number of tokens we need
# need improvement since we select training rows randomly
def max_submission_size(t, max_size):
  idf = data[data["QuestionId"] == t['QuestionId']]
  for i in range(1,len(idf)-1):
    mp = generate_fewshot_prompt_for_row(t,i)
    if num_tokens_from_string(mp, "cl100k_base") >= max_size:
      return i-1
  return None

In [ ]:
# 8196 tokens limit for gpt4, select a maximum submission size for each row
lst = []
for i, r in tqdm(data1.iterrows(), total=len(data1)):
  s_size = max_submission_size(r, 6500)
  lst.append(generate_fewshot_prompt_for_row(r,s_size))

data1['4_few_mp'] = lst
data1.head()

100%|██████████| 300/300 [04:22<00:00,  1.14it/s]


,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema,4_few_mp,num
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",...,xs2262,NaN,"SELECT track_name, artist_name, album_name\r\n...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,16322
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",...,ty2394,NaN,"SELECT artist_id, album_id, SUM(time) AS total...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,11975
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",...,bak8119,NaN,"SELECT ProductID, ProductNameFROM Products WHE...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,9889
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,...,lj2371,NaN,\r\nSELECT *\r\nFROM track\r\nWHERE artist_id ...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,8637
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",...,fc2187,NaN,"SELECT FirstName, LastName, HireDate FROM Empl...",True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,11330


In [ ]:
data1['num'] = data1['4_few_mp'].apply(num_tokens_from_string)
data1.head()
print(data1['num'].max()) # print the maximum number of tokens among all mega prompts, which is < 8196

7415


In [ ]:
# dealing with rate limit of GPT4
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT4_generation_with_backoff(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
import time
# further control rate limit for GPT-4
rate_limit_per_minute = 40000
delay = 60.0 / rate_limit_per_minute

def delayed_completion(p):
  time.sleep(delay)
  return GPT4_generation_with_backoff(p)

In [ ]:
tqdm.pandas()
# generate GPT4 answer for each mega prompt as a new column of JSON objects
data1['4_answer'] = data1['4_few_mp'].progress_apply(delayed_completion)

100%|██████████| 300/300 [41:28<00:00,  8.30s/it]


In [ ]:
# parse grade and feedback from json object
def extract_grade(js):
  try:
    js = json.loads(js)
    return js["grade"]
  except json.JSONDecodeError:
    return ''

def extract_feedback(js):
  try:
    js = json.loads(js)
    return js["feedback"]
  except json.JSONDecodeError:
    return ''

In [ ]:
# Extract grade and feedback as new columns
data1['4_grade'] = data1['4_answer'].apply(extract_grade)
data1['4_feedback'] = data1['4_answer'].apply(extract_feedback)
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,isQuery,Database,full_score,feedback,schema,4_few_mp,num,4_answer,4_grade,4_feedback
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,5675,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,6259,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",...,True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,6504,"{""grade"": ""10"", ""feedback"": ""Your answer is co...",10,Your answer is correct. Good job on using the ...
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,6535,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",...,True,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,6362,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query is correct and it me...


In [ ]:
data1['4_grade'].value_counts()

10/10      141
10         102
7.5/10      10
9.5/10       9
8.0          8
7.0          6
5.0          4
9.5          4
7/10         3
8.5/10       2
0.0          2
2.5/10       2
0/10         1
0.0/10       1
0.0/1.0      1
5/10         1
9.0          1
5.5/10       1
0.7/10       1
Name: 4_grade, dtype: int64

In [ ]:
# Repeat the same process for GPT3.5-turbo mega prompt and responses
lst = []
for i, r in tqdm(data1.iterrows(), total=len(data1)):
  s_size = max_submission_size(r, 2500) # max # of tokens = 4096
  lst.append(generate_fewshot_prompt_for_row(r,s_size))

data1['3_few_mp'] = lst

data1['num'] = data1['3_few_mp'].apply(num_tokens_from_string)
print(data1['num'].max())
data1.head()

100%|██████████| 300/300 [00:49<00:00,  6.06it/s]


3161


,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,Database,full_score,feedback,schema,4_few_mp,num,4_answer,4_grade,4_feedback,3_few_mp
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",...,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2496,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...,\n a. We are working with the music datab...
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",...,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2529,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...,\n a. We are working with the music datab...
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",...,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,2468,"{""grade"": ""10"", ""feedback"": ""Your answer is co...",10,Your answer is correct. Good job on using the ...,\n a. We are working with the ['northwind...
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,...,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2435,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...,\n a. We are working with the music datab...
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",...,"['northwind', 'flights']",5.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,2386,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query is correct and it me...,\n a. We are working with the ['northwind...


In [ ]:
tqdm.pandas()
data1['3_answer'] = data1['3_few_mp'].progress_apply(GPT3_generation_with_backoff)

100%|██████████| 300/300 [23:29<00:00,  4.70s/it]


In [ ]:
data1['3_grade'] = data1['3_answer'].apply(extract_grade)
data1['3_feedback'] = data1['3_answer'].apply(extract_feedback)
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,schema,4_few_mp,num,4_answer,4_grade,4_feedback,3_few_mp,3_answer,3_grade,3_feedback
0,1838,"Databases for Business Analytics, MSQM SU23",277478,Assignment 3: Joins,301296,1,5119721,"List all the track names, the corresponding al...",Long Answer Question,"select t.track_name, a.artist_name, b.album_na...",...,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2496,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...,\n a. We are working with the music datab...,"{""grade"": ""1.0"", ""feedback"": ""The query provid...",1.0,The query provided by the student is correct a...
1,5850,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 4: Aggregations,302626,1,5119732,"List the IDs of the albums, where the total al...",Long Answer Question,"SELECT artist_id, album_id, SUM(time) AS track...",...,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2529,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...,\n a. We are working with the music datab...,"{""grade"": ""0.7"", ""feedback"": ""The query is mis...",0.7,The query is missing one of the album keys in ...
2,6124,"Databases for Business Analytics, TechMBA May'23",284275,Final Exam,302604,1,4969707,"From the Products table, we want to see the Pr...",Long Answer Question,"select ProductID, ProductName\r\nfrom Products...",...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,2468,"{""grade"": ""10"", ""feedback"": ""Your answer is co...",10,Your answer is correct. Good job on using the ...,\n a. We are working with the ['northwind...,"{""grade"": ""5.0"", ""feedback"": ""Your query is co...",5.0,Your query is correct and returns the desired ...
3,4928,"Databases for Business Analytics, MSBA Jun'23",284280,Assignment 2C: Filtering Queries,302620,1,5119676,"Find the tracks for the artist with id 3, from...",Long Answer Question,SELECT track_name\r\nFROM track\r\nWHERE artis...,...,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2435,"{""grade"": ""10/10"", ""feedback"": ""Your answer is...",10/10,Your answer is correct. You have successfully ...,\n a. We are working with the music datab...,"{""grade"": ""10"", ""feedback"": ""The student's ans...",10,The student's answer is correct. They correctl...
4,148,"Databases for Business Analytics, Winter 2023",251683,Final Exam,283453,1,4969706,"We would like to see just the FirstName, LastN...",Long Answer Question,"USE northwind;\r\nSELECT FirstName, LastName, ...",...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,2386,"{""grade"": ""10/10"", ""feedback"": ""Great job! You...",10/10,Great job! Your SQL query is correct and it me...,\n a. We are working with the ['northwind...,"{""grade"": ""5.0"", ""feedback"": ""The query provid...",5.0,The query provided by the student is correct a...


In [ ]:
data1['3_grade'].value_counts()

10          59
1.0         48
7.5         34
10/10       29
0.8         18
0.9         16
8.5         16
5.0         11
9.5         11
0.7         11
9.0         10
10.0         8
8.5/10.0     5
2.0          5
7.5/10       4
1.0/11.0     3
1.0/2.0      2
1.5          2
7.5/10.0     2
0.95         2
8.0          2
0.4          1
8.0/10.0     1
Name: 3_grade, dtype: int64

In [ ]:
data1.to_csv('few_completions.csv')